In [1]:
import pandas as pd
import os
location = str(os.getcwd())

In [2]:
f = (location[:-8] + "Articles\\articles_cleaned.csv")
articles = pd.read_csv(f)
articles = articles.to_dict("index")

In [3]:
f = (location[:-8] + "Tweets\\tweets_cleaned.csv")
tweets = pd.read_csv(f)
tweets = tweets.to_dict("index")

In [4]:
locations = ["IUB", "UMD", "UMN", "Stanford", "Wisconsin"]

from datetime import date, timedelta
sdate = date(2000,1,1)   # start date
edate = date(2022,1,2)   # end date

unformatted_dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d').tolist()

dates = []
for item in unformatted_dates:
    y = str(item.year)
    if (item.day) < 10:
        d = "0"+str(item.day)
    else:
        d = str(item.day)
    if (item.day) < 10:
        m = "0"+str(item.month)
    else:
        m = str(item.month)
    dates.append(m +"/"+ d +"/"+ y)
print(len(dates))

8037


In [5]:
by_date = []

for location in locations:
    print(location)
    k = 0
    for date in dates:
        date_dict = {}
        date_dict["location"] = location
        date_dict["date"] = date
        tweet_count = 0
        article_count = 0

        for i in range(len(tweets)):
            tweet = tweets[i]
            if date == tweet["created_at"] and tweet["location"] == location:
                tweet_count += 1

        for j in range(len(articles)):
            article = articles[j]
            if date == article["date"] and article["location"] == location:
                article_count += 1
        
        if k == 0:
            date_dict["article_coverage"] = article_count
        else:
            date_dict["article_coverage"] = (by_date[k-1]["article_coverage"]*(14/15))+article_count
            
        if k == 0:
            date_dict["tweet_coverage"] = tweet_count
        else:
            date_dict["tweet_coverage"] = (by_date[k-1]["tweet_coverage"]*(2/3))+tweet_count
            
        date_dict["tweet_count"] = tweet_count
        date_dict["article_count"] = article_count

        by_date.append(date_dict)
        k += 1
                                           
pd.DataFrame(by_date)

IUB
UMD
UMN
Stanford
Wisconsin


,location,date,article_coverage,tweet_coverage,tweet_count,article_count
0,IUB,01/01/2000,0.000000,0.000000,0,0
1,IUB,01/02/2000,0.000000,0.000000,0,0
2,IUB,01/03/2000,0.000000,0.000000,0,0
3,IUB,01/04/2000,0.000000,0.000000,0,0
4,IUB,01/05/2000,0.000000,0.000000,0,0
...,...,...,...,...,...,...
40180,Wisconsin,12/28/2021,1.184388,0.147788,0,0
40181,Wisconsin,12/29/2021,1.105428,0.098525,0,0
40182,Wisconsin,12/30/2021,1.031733,0.065683,0,0
40183,Wisconsin,12/31/2021,0.962951,0.043789,0,0


In [6]:
pd.DataFrame(by_date).to_csv('counts_by_day.csv')

Average for each day across all years to check for seasonality (Average between 2011 and end of 2021)

In [3]:
location = str(os.getcwd())
f = (location + "\counts_by_day.csv")
by_dates = pd.read_csv(f)
by_dates = by_dates.to_dict("recirds") 

In [4]:
from datetime import date, timedelta

start_date = date(1999, 1, 1) 
end_date = date(1999, 12, 31)    # perhaps date.now()

delta = end_date - start_date   # returns timedelta
dates = []
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    dates.append(day)
len(dates)

365

In [7]:
df = []
locations = ["IUB", "UMD", "UMN", "Stanford", "Wisconsin"]
for location in locations:
    for date in dates:
        date = str(date)
        comp_day = date.split("-")[1] + "/" +  date.split("-")[2]

        total_tweets = 0
        total_articles = 0
        total_coverage = 0
        for day in by_dates:
            full = str(day["date"])
            if str(full) != "nan" and int(full.split("/")[2]) > 2010 and int(full.split("/")[2]) < 2022:
                m = full.split("/")[0]
                if int(m) < 10:
                    m = "0"+m
                d = full.split("/")[1]
                if int(d) < 10:
                    d = "0"+d
                md = m + "/" + d
                if comp_day == md and location == day["location"]:
                    total_articles += day["tweet_count"]
                    total_tweets += day["article_count"]
                    total_coverage += day["coverage_value"]
                    
        average_tweets = total_tweets/11
        average_articles = total_articles/11
        average_coverage = total_coverage/11
        df.append({"date":date, "average_tweets": average_tweets, "average_articles": average_articles, "location": location, "coverage": average_coverage})

pd.DataFrame(df)

,date,average_tweets,average_articles,location,coverage
0,1999-01-01,0.000000,0.000000,IUB,0.000000
1,1999-01-02,0.000000,0.000000,IUB,0.000000
2,1999-01-03,0.000000,0.000000,IUB,0.000000
3,1999-01-04,0.000000,0.000000,IUB,0.000000
4,1999-01-05,0.000000,0.000000,IUB,0.000000
...,...,...,...,...,...
1820,1999-12-27,0.090909,0.363636,Wisconsin,1.830223
1821,1999-12-28,0.000000,0.363636,Wisconsin,1.623359
1822,1999-12-29,0.000000,0.272727,Wisconsin,1.599984
1823,1999-12-30,0.090909,0.636364,Wisconsin,1.584227


In [8]:
pd.DataFrame(df).to_csv('yearly_averages.csv')